In [ ]:
# ================================
# Differential Privacy – Simple Queries
# ================================
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

def classify_utility(rel_err_pct: float) -> str:
    if rel_err_pct < 5:
        return "Good"
    elif rel_err_pct < 15:
        return "Moderate"
    else:
        return "Poor"

def generalize_age_to_3bins(age_series):
    def to_bucket(val):
        if pd.isna(val) or str(val).strip().upper() in {"?", "UNKNOWN", "NULL"}:
            return val
        s = str(val).strip(" []()")
        try:
            low, high = map(int, s.split("-", 1))
        except:
            return val
        if high <= 30:
            return "0-30"
        elif high <= 60:
            return "30-60"
        else:
            return ">60"
    return age_series.map(to_bucket)

# ------------------------------------------------
# Parameters
# ------------------------------------------------
slices = {
    "25k":  "diabetic_data_25k.csv",
    "50k":  "diabetic_data_50k.csv",
    "75k":  "diabetic_data_75k.csv",
    "full": "diabetic_data_final.csv"
}

epsilons = [2.0, 5.0, 8.0]


epsilon_weights = {
    "Age-3bin counts":             0.10,
    "Race counts":                 0.10,
    "Gender×AdmType counts":       0.20,
    "Avg # meds by age3bin":       0.30,
    "Readmit(<30%) by race":       0.30
}

# Normalize weights to compute relative ε allocation
total_weight = sum(epsilon_weights.values())
query_epsilons = {q: w / total_weight for q, w in epsilon_weights.items()}

timing_records = []
utility_records = []

# ------------------------------------------------
# DP Execution
# ------------------------------------------------
for slice_label, path in slices.items():
    df = pd.read_csv(path, keep_default_na=False)
    df["age3"] = generalize_age_to_3bins(df["age"])

    true_q1 = df["age3"].value_counts().reindex(["0-30", "30-60", ">60"], fill_value=0)
    true_q2 = df["race"].value_counts().sort_index()
    true_q3 = df.groupby(["gender", "admission_type"]).size().sort_index()
    true_q4 = df.groupby("age3")["num_medications"].mean().reindex(["0-30", "30-60", ">60"], fill_value=0)
    true_q5 = df.groupby("race")["readmitted"].apply(lambda s: (s == "<30").mean() * 100).sort_index()

    for eps in epsilons:
        print(f"Processing slice={slice_label}, ε={eps}")

        ### Query 1: Age-3bin counts
        ε1 = eps * query_epsilons["Age-3bin counts"]
        scale1 = 2.0 / ε1
        t0 = time.perf_counter()
        noisy_q1 = true_q1 + np.random.laplace(loc=0, scale=scale1, size=len(true_q1))
        t1 = time.perf_counter()
        err1 = np.abs(noisy_q1 - true_q1) / true_q1.replace(0, np.nan) * 100
        timing_records.append({"Slice": slice_label, "Epsilon": eps, "Query": "Age-3bin counts", "Time_s": t1 - t0})
        utility_records.append({"Slice": slice_label, "Epsilon": eps, "Query": "Age-3bin counts", "RelErr(%)": err1.mean(), "Utility": classify_utility(err1.mean())})

        ### Query 2: Race counts
        ε2 = eps * query_epsilons["Race counts"]
        scale2 = 2.0 / ε2
        t0 = time.perf_counter()
        noisy_q2 = true_q2 + np.random.laplace(loc=0, scale=scale2, size=len(true_q2))
        t1 = time.perf_counter()
        err2 = np.abs(noisy_q2 - true_q2) / true_q2.replace(0, np.nan) * 100
        timing_records.append({"Slice": slice_label, "Epsilon": eps, "Query": "Race counts", "Time_s": t1 - t0})
        utility_records.append({"Slice": slice_label, "Epsilon": eps, "Query": "Race counts", "RelErr(%)": err2.mean(), "Utility": classify_utility(err2.mean())})

        ### Query 3: Gender×AdmType
        ε3 = eps * query_epsilons["Gender×AdmType counts"]
        scale3 = 2.0 / ε3
        t0 = time.perf_counter()
        noisy_q3 = true_q3 + np.random.laplace(loc=0, scale=scale3, size=len(true_q3))
        t1 = time.perf_counter()
        err3 = np.abs(noisy_q3 - true_q3) / true_q3.replace(0, np.nan) * 100
        timing_records.append({"Slice": slice_label, "Epsilon": eps, "Query": "Gender×AdmType counts", "Time_s": t1 - t0})
        utility_records.append({"Slice": slice_label, "Epsilon": eps, "Query": "Gender×AdmType counts", "RelErr(%)": err3.mean(), "Utility": classify_utility(err3.mean())})

        ### Query 4: Avg # meds by age3bin
        ε4 = eps * query_epsilons["Avg # meds by age3bin"]
        M_max = df["num_medications"].max()
        min_count = df["age3"].value_counts().min() or 1
        Δf4 = 2.0 * M_max / min_count
        scale4 = Δf4 / ε4
        t0 = time.perf_counter()
        noisy_q4 = true_q4 + np.random.laplace(loc=0, scale=scale4, size=len(true_q4))
        t1 = time.perf_counter()
        err4 = np.abs(noisy_q4 - true_q4) / true_q4.replace(0, np.nan) * 100
        timing_records.append({"Slice": slice_label, "Epsilon": eps, "Query": "Avg # meds by age3bin", "Time_s": t1 - t0})
        utility_records.append({"Slice": slice_label, "Epsilon": eps, "Query": "Avg # meds by age3bin", "RelErr(%)": err4.mean(), "Utility": classify_utility(err4.mean())})

        ### Query 5: Readmit% by race
        ε5 = eps * query_epsilons["Readmit(<30%) by race"]
        min_race_n = df["race"].value_counts().min() or 1
        Δf5 = (2 / min_race_n) * 100
        scale5 = Δf5 / ε5
        t0 = time.perf_counter()
        noisy_q5 = true_q5 + np.random.laplace(loc=0, scale=scale5, size=len(true_q5))
        t1 = time.perf_counter()
        err5 = np.abs(noisy_q5 - true_q5) / true_q5.replace(0, np.nan) * 100
        timing_records.append({"Slice": slice_label, "Epsilon": eps, "Query": "Readmit(<30%) by race", "Time_s": t1 - t0})
        utility_records.append({"Slice": slice_label, "Epsilon": eps, "Query": "Readmit(<30%) by race", "RelErr(%)": err5.mean(), "Utility": classify_utility(err5.mean())})

# ------------------------------------------------
# Build summary tables
# ------------------------------------------------
df_time = pd.DataFrame(timing_records)
df_util = pd.DataFrame(utility_records)

print("\n--- Timing Summary ---")
display(df_time)

print("\n--- Utility Summary (Pivot like t-closeness) ---")
# Create the wide utility summary identical in style to t-closeness:
summary_pivot = (
    df_util
      .pivot_table(
          index=["Slice", "Query"],
          columns="Epsilon",
          values=["RelErr(%)", "Utility"],
          aggfunc="first"
      )
      .round(2)
)
summary_pivot.columns.name = None
summary_pivot = summary_pivot.reset_index()
display(summary_pivot)

# Optional: save as CSV
df_time.to_csv("dp_timing_simple_summary.csv", index=False)
summary_pivot.to_csv("dp_simple_query_utility_summary.csv", index=False)


In [ ]:
# =============================================================================
# Differential Privacy for Complex Queries
# — allocate ε across queries by weight, save dp_timing_summary & dp_utility_summary
# =============================================================================

import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

# -----------------------------------------------------------------------------
# Helper Functions
# -----------------------------------------------------------------------------
def classify_utility(rel_err_pct: float) -> str:
    if rel_err_pct < 5:
        return "Good"
    elif rel_err_pct < 15:
        return "Moderate"
    else:
        return "Poor"

def generalize_age_to_3bins(age_series: pd.Series) -> pd.Series:
    def to_bucket(val):
        if pd.isna(val) or str(val).strip().upper() in {"?", "UNKNOWN", "NULL"}:
            return val
        s = str(val).strip(" []()")
        try:
            low, high = map(int, s.split("-", 1))
        except:
            return val
        if high <= 30:
            return "0-30"
        if high <= 60:
            return "30-60"
        return ">60"
    return age_series.map(to_bucket)

def dp_query(true_ser: pd.Series, Δf: float, eps_q: float,
             slice_label: str, eps: float, query_name: str,
             timing_records: list, utility_records: list):
    """
    Add Laplace noise (scale = Δf/eps_q) to true_ser,
    record timing & compute utility.
    """
    scale = Δf / eps_q
    t0    = time.perf_counter()
    noisy = true_ser.map(lambda v: float(v) + np.random.laplace(0, scale))
    t1    = time.perf_counter()

    # record timing
    timing_records.append({
        "Slice":   slice_label,
        "Epsilon": eps,
        "Query":   query_name,
        "Time_s":  t1 - t0
    })

    # compute relative error & utility
    comp = pd.DataFrame({
        "true":  true_ser,
        "noisy": noisy.reindex(true_ser.index).fillna(0)
    })
    comp["rel_err_pct"] = np.where(
        comp["true"] == 0,
        np.nan,
        (comp["noisy"] - comp["true"]).abs() / comp["true"] * 100
    )
    mean_err = comp["rel_err_pct"].dropna().mean()
    util     = classify_utility(mean_err if not np.isnan(mean_err) else 0.0)

    utility_records.append({
        "Slice":     slice_label,
        "Epsilon":   eps,
        "Query":     query_name,
        "RelErr(%)": mean_err,
        "Utility":   util
    })


# -----------------------------------------------------------------------------
# 1) Define dataset slices & global ε values
# -----------------------------------------------------------------------------
slices   = {
    "25k":  "diabetic_data_25k.csv",
    "50k":  "diabetic_data_50k.csv",
    "75k":  "diabetic_data_75k.csv",
    "full": "diabetic_data_final.csv"
}
epsilons = [2.0, 5.0, 8.0]

# -----------------------------------------------------------------------------
# 2) Define per-query ε‐allocation weights (must sum to 1.0)
# -----------------------------------------------------------------------------
query_weights = {
    "Avg LabCtrls by age3×adm":     0.30,
    "Avg LabCtrls by diag2×adm":    0.20,
    "Avg Meds by race×adm":         0.20,
    "Avg Meds by diag2×gender":     0.20,
    "Readmit% by diag1×gender":     0.10,
   
    
    
}
assert abs(sum(query_weights.values()) - 1.0) < 1e-6, "Weights must sum to 1.0"

# -----------------------------------------------------------------------------
# 3) Prepare data structures to collect results
# -----------------------------------------------------------------------------
timing_records = []
utility_records = []

# -----------------------------------------------------------------------------
# 4) Core DP Pipeline: loop over slices & global epsilons
# -----------------------------------------------------------------------------
for slice_label, path in slices.items():
    print(f"\n=== Processing slice '{slice_label}' ({path}) ===")
    df = pd.read_csv(path, keep_default_na=False)
    df["age3"] = generalize_age_to_3bins(df["age"])

    # precompute max‐ranges & minimum nonzero group‐sizes
    max_lab  = df["num_lab_procedures"].max()
    max_meds = df["num_medications"].max()

    def min_group_size(cols):
        cnt = df.groupby(cols).size()
        return cnt[cnt > 0].min() if not cnt.empty else 1

    m1 = min_group_size(["age3","admission_type"])
    m2 = min_group_size(["diagnoses_2","admission_type"])
    m3 = min_group_size(["race","admission_type"])
    m4 = min_group_size(["diagnoses_2","gender"])
    m5 = min_group_size(["diagnoses_1","gender"])
    
    
    

    # compute true-answer series
    q1 = df.groupby(["age3","admission_type"])["num_lab_procedures"]\
           .mean().sort_index()
    q2 = df.groupby(["diagnoses_2","admission_type"])["num_lab_procedures"]\
           .mean().sort_index()
    q3 = df.groupby(["race","admission_type"])["num_medications"]\
           .mean().sort_index()
    q4 = df.groupby(["diagnoses_2","gender"])["num_medications"]\
           .mean().sort_index()
    q5 = df.groupby(["diagnoses_1","gender"])["readmitted"]\
           .apply(lambda s: (s=="<30").mean()*100).sort_index()
    
    
    

    # loop over global epsilons
    for eps in epsilons:
        print(f"  -- ε = {eps}")
        # allocate ε to each query
        eps_alloc = {q: eps * w for q,w in query_weights.items()}

        dp_query(
            true_ser=q1,
            Δf      = 2 * max_lab / m1,
            eps_q   = eps_alloc["Avg LabCtrls by age3×adm"],
            slice_label=slice_label,
            eps      = eps,
            query_name="Avg LabCtrls by age3×adm",
            timing_records=timing_records,
            utility_records=utility_records
        )

        dp_query(
            true_ser=q2,
            Δf      = 2 * max_lab / m2,
            eps_q   = eps_alloc["Avg LabCtrls by diag2×adm"],
            slice_label=slice_label,
            eps      = eps,
            query_name="Avg LabCtrls by diag2×adm",
            timing_records=timing_records,
            utility_records=utility_records
        )
        dp_query(
            true_ser=q3,
            Δf      = 2 * max_meds / m3,
            eps_q   = eps_alloc["Avg Meds by race×adm"],
            slice_label=slice_label,
            eps      = eps,
            query_name="Avg Meds by race×adm",
            timing_records=timing_records,
            utility_records=utility_records
        )
        dp_query(
            true_ser=q4,
            Δf      = 2 * max_meds / m4,
            eps_q   = eps_alloc["Avg Meds by diag2×gender"],
            slice_label=slice_label,
            eps      = eps,
            query_name="Avg Meds by diag2×gender",
            timing_records=timing_records,
            utility_records=utility_records
        )

        dp_query(
            true_ser=q5,
            Δf      = 2 * 100 / m5,
            eps_q   = eps_alloc["Readmit% by diag1×gender"],
            slice_label=slice_label,
            eps      = eps,
            query_name="Readmit% by diag1×gender",
            timing_records=timing_records,
            utility_records=utility_records
        )

        

        
        

# -----------------------------------------------------------------------------
# 5) Build DataFrames & Summaries, save to CSV
# -----------------------------------------------------------------------------
df_timing  = pd.DataFrame(timing_records)
df_utility = pd.DataFrame(utility_records)

# summary of total DP time per (Slice, Epsilon) — unchanged
dp_timing_summary = (
    df_timing
    .groupby(["Slice","Epsilon"], as_index=False)["Time_s"]
    .sum()
)
dp_timing_summary.to_csv("dp_timing_complex_summary.csv", index=False)

# utility summary in the SAME format as t-closeness (wide pivot by ε)
summary_pivot = (
    df_utility
      .pivot_table(
          index=["Slice", "Query"],
          columns="Epsilon",
          values=["RelErr(%)", "Utility"],
          aggfunc="first"
      )
      .round(2)
)
summary_pivot.columns.name = None
summary_pivot = summary_pivot.reset_index()
summary_pivot.to_csv("dp_complex_query_utility_summary.csv", index=False)

print("\n→ Saved dp_timing_complex_summary.csv and dp_complex_query_utility_summary.csv\n")

# -----------------------------------------------------------------------------
# 6) (Optional) Display raw & summary tables
# -----------------------------------------------------------------------------
print("=== Raw Timing ===")
display(df_timing)

print("=== Raw Utility ===")
display(df_utility)

print("=== Timing Summary ===")
display(dp_timing_summary)

print("=== Utility Summary (Pivot like t-closeness) ===")
display(summary_pivot)

# -----------------------------------------------------------------------------
# 7) Plot Total DP Time per Slice (stacked by ε)
# -----------------------------------------------------------------------------
pivot = dp_timing_summary.pivot(index="Slice", columns="Epsilon", values="Time_s")\
                         .loc[["25k","50k","75k","full"]]
plt.figure(figsize=(8,5))
pivot.plot(kind="bar")
plt.xlabel("Dataset Slice")
plt.ylabel("Total DP Runtime (s)")
plt.title("Total DP Time by Slice & ε")
plt.xticks(rotation=0)
plt.legend(title="ε")
plt.tight_layout()
plt.show()
